In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

In [2]:
df = pd.read_csv("amazon_products_sales_data_uncleaned.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     42675 non-null  object
 1   rating                    41651 non-null  object
 2   number_of_reviews         41651 non-null  object
 3   bought_in_last_month      39458 non-null  object
 4   current/discounted_price  30926 non-null  object
 5   price_on_variant          42675 non-null  object
 6   listed_price              42675 non-null  object
 7   is_best_seller            42675 non-null  object
 8   is_sponsored              42675 non-null  object
 9   is_couponed               42675 non-null  object
 10  buy_box_availability      28022 non-null  object
 11  delivery_details          30955 non-null  object
 12  sustainability_badges     3408 non-null   object
 13  image_url                 42675 non-null  object
 14  product_url           

In [4]:
df.drop_duplicates()

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6 out of 5 stars,375,300+ bought in past month,89.68,basic variant price: 2.4GHz,$159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3 out of 5 stars,"2,457",6K+ bought in past month,9.99,basic variant price: nan,$15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6 out of 5 stars,"3,044",2K+ bought in past month,314.00,basic variant price: nan,$349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6 out of 5 stars,"35,882",10K+ bought in past month,NaN,basic variant price: $162.24,No Discount,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8 out of 5 stars,"28,988",10K+ bought in past month,NaN,basic variant price: $72.74,No Discount,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42670,"Elgato 4K Pro, Internal Capture Card: 8K60 Pas...",5.0 out of 5 stars,1,100+ bought in past month,195.99,basic variant price: nan,No Discount,No Badge,Organic,No Coupon,NaN,"FREE delivery Thu, Sep 4Or fastest delivery Tu...",NaN,https://m.media-amazon.com/images/I/51KCB+egEs...,/Elgato-4K-Pro-Internal-Capture/dp/B0DLR3WQWR/...,2025-08-30 19:56:33
42671,"Arlo Essential Spotlight Camera, Wireless Secu...",4.2 out of 5 stars,20,200+ bought in past month,89.99,basic variant price: $25.00 off coupon applied,No Discount,No Badge,Organic,Save $25.00 with coupon,Add to cart,"FREE delivery Thu, Sep 4Or fastest delivery Mo...",NaN,https://m.media-amazon.com/images/I/51jV+o1LZE...,/Arlo-Essential-Spotlight-Camera-Surveillance/...,2025-08-30 19:56:33
42672,"GIGABYTE - AORUS FO32U2-32"" QD OLED Gaming Mon...",4.3 out of 5 stars,57,50+ bought in past month,899.99,basic variant price: Lowest price in 30 days,"$1,099.99",Save 18%,Organic,No Coupon,Add to cart,"FREE delivery Thu, Sep 4Or fastest delivery We...",NaN,https://m.media-amazon.com/images/I/71ySPkNLkG...,/GIGABYTE-FO32U2-32-3840x2160-240Hz-FreeSync-A...,2025-08-30 19:56:33
42673,Monoprice XLR Male to 1/4-Inch TRS Male Cable ...,4.7 out of 5 stars,"7,102",500+ bought in past month,10.39,basic variant price: nan,$15.98,No Badge,Organic,No Coupon,Add to cart,"FREE delivery Thu, Sep 4 on $35 of items shipp...",NaN,https://m.media-amazon.com/images/I/411c0JFJ79...,/Monoprice-XLR-Male-4-Inch-Cable/dp/B001UJEKZ6...,2025-08-30 19:56:33


In [5]:
# First, remove any non-numeric characters and handle empty strings
df['rating'] = df['rating'].str.extract(r'([\d\.]+)').astype(float)

# First, remove any non-numeric characters, handle empty strings, and convert to int
df['number_of_reviews'] = df['number_of_reviews'].str.replace(',', '').str.extract(r'(\d+)')
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'], errors='coerce').fillna(0).astype(int)


In [6]:
df['collected_at'] = pd.to_datetime(df['collected_at'])


In [7]:
df["is_best_seller"].value_counts()

is_best_seller
No Badge             40814
Amazon's               744
Limited time deal      409
Best Seller            275
Save 30%               125
Ends in                 97
Save 10%                61
Save 18%                57
Save 17%                51
Save 12%                26
Save 9%                 14
Save 77%                 2
Name: count, dtype: int64

In [8]:
def clean_sales(value):
    if pd.isna(value):
        return 0
    
    value = str(value).lower()
    
    # Try to handle the case with 'k' notation (e.g., '5k+')
    if 'k' in value:
        try:
            # Extract number, remove 'k', multiply by 1000
            number = value.split('k')[0].replace('+', '')
            return int(float(number) * 1000)
        except ValueError:
            # If conversion fails, return 0
            return 0
    
    # Try to handle the case with 'bought' notation (e.g., '300+ bought')
    elif 'bought' in value:
        try:
            # Standard number like "300+ bought"
            number = value.split('+')[0]
            return int(number)
        except ValueError:
            # If conversion fails, return 0
            return 0
    
    # Try to convert directly to int if it's a simple number
    else:
        try:
            return int(float(value))
        except ValueError:
            # If all conversions fail, return 0
            return 0

# Apply the function to the column
df['bought_in_last_month'] = df['bought_in_last_month'].apply(clean_sales)

In [9]:
def clean_price(price_str):
    if pd.isna(price_str):
        return np.nan
    price_str = str(price_str).strip()
    dollars = re.findall(r'\$?(\d+(?:\.\d{2})?)', price_str)
    if dollars:
        return float(dollars[0])
    return np.nan

df['price_on_variant'] = df['price_on_variant'].apply(clean_price)

In [10]:
df['listed_price'] = df['listed_price'].replace({'No Discount': np.nan, 'No Badge': np.nan})
df['listed_price'] = df['listed_price'].str.replace('$', '', regex=False)
df['listed_price'] = pd.to_numeric(df['listed_price'], errors='coerce')

In [11]:
df['is_best_seller'] = np.where(df['is_best_seller'] == 'No Badge',False,True)
df['is_sponsored'] = np.where(df['is_sponsored'] == 'Sponsored',True,False)
df['is_couponed'] = np.where(df['is_couponed'] == 'No Coupon',False,True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   title                     42675 non-null  object        
 1   rating                    41651 non-null  float64       
 2   number_of_reviews         42675 non-null  int64         
 3   bought_in_last_month      42675 non-null  int64         
 4   current/discounted_price  30926 non-null  float64       
 5   price_on_variant          21298 non-null  float64       
 6   listed_price              11673 non-null  float64       
 7   is_best_seller            42675 non-null  bool          
 8   is_sponsored              42675 non-null  bool          
 9   is_couponed               42675 non-null  bool          
 10  buy_box_availability      28022 non-null  object        
 11  delivery_details          30955 non-null  object        
 12  sustainability_bad

In [13]:
df['current/discounted_price'] = df['current/discounted_price'].str.replace(',', '').astype('float64')

In [14]:
df.to_csv('amazon_products_sales_data_cleaned.csv', index=False)